<a href="https://colab.research.google.com/github/ins-uni-bonn/mllab-project-marcolparnell/blob/mateusz/cnn_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_tuner
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
# from read_cifar import load_cifar
import numpy as np
import keras_tuner

In [ ]:
from read_cifar import load_cifar
training_data, test_data = load_cifar()

In [ ]:
# here we normalzie the data
training_data["data"] = training_data["data"] / 255

mean_per_channel = np.mean(training_data["data"], axis=(0, 1, 2))
std_per_channel = np.std(training_data["data"], axis=(0, 1, 2))
training_data["data"] = (training_data["data"] - mean_per_channel)/std_per_channel

test_data["data"] = test_data["data"] / 255
mean_per_channel = np.mean(test_data["data"], axis=(0, 1, 2))
std_per_channel = np.std(test_data["data"], axis=(0, 1, 2))
test_data["data"] = (test_data["data"] - mean_per_channel)/std_per_channel

In [ ]:
# first approach to the pedestrian model
cifar_cnn = keras.Sequential()

cifar_cnn.add(layers.Input(shape=(32,32,3)))

# Data augmentation
cifar_cnn.add(layers.RandomFlip(mode="horizontal_and_vertical"))
cifar_cnn.add(layers.RandomRotation(0.15))
# cifar_cnn.add(layers.RandomZoom(0.15, 0.15))

# First conv block
cifar_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
cifar_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cifar_cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
cifar_cnn.add(layers.Dropout(0.25))

# # Second conv block
cifar_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cifar_cnn.add(layers.BatchNormalization())
cifar_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cifar_cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
cifar_cnn.add(layers.Dropout(0.25))

# Fully connected layers
cifar_cnn.add(layers.Flatten())
cifar_cnn.add(layers.Dense(512, activation='relu'))
cifar_cnn.add(layers.Dropout(0.5))
cifar_cnn.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

cifar_cnn.summary()

callbac_cifra_cnn = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

cifar_cnn.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history_cifar_cnn = cifar_cnn.fit(
    training_data["data"],
    training_data["labels"],
    batch_size = 1024,
    epochs=100,
    validation_split=0.1,
    callbacks=[callbac_cifra_cnn]
    )

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_6 (RandomFlip)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_7               │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,168,618 (8.27 MB)

 Trainable params: 2,168,490 (8.27 MB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 105ms/step - accuracy: 0.1281 - loss: 3.9402 - val_accuracy: 0.2384 - val_loss: 2.2920
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.2079 - loss: 2.0994 - val_accuracy: 0.2642 - val_loss: 2.2621
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.2544 - loss: 1.9595 - val_accuracy: 0.3244 - val_loss: 2.2104
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.2874 - loss: 1.8769 - val_accuracy: 0.3132 - val_loss: 2.1623
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - accuracy: 0.3162 - loss: 1.8128 - val_accuracy: 0.4054 - val_loss: 2.0962
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.3278 - loss: 1.7798 - val_accuracy: 0.4054 - val_loss: 2.0048
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.3429 - loss: 1.7368 - val_accuracy: 0.4368 - val_loss: 1.8948
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.3527 - loss: 1.7102 - val_accuracy: 0

In [ ]:
cifar_cnn.evaluate(test_data["data"], test_data["labels"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6933 - loss: 0.8912


[0.894550085067749, 0.6935999989509583]

In [ ]:
def build_cnn(hp):
  # first approach to the pedestrian model
  model = keras.Sequential()

  model.add(layers.Input(shape=(32,32,3)))

  # Data augmentation
  model.add(layers.RandomFlip(mode="horizontal_and_vertical"))
  model.add(layers.RandomRotation(0.15))
  # model.add(layers.RandomZoom(0.15, 0.15))

  # First conv block
  model.add(layers.Conv2D(filters=hp.Int("conv_1", min_value=16, max_value = 48, step = 8), kernel_size = (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
  model.add(layers.Conv2D(filters=hp.Int("conv_2", min_value=16, max_value = 48, step = 8), kernel_size = (3, 3), activation='relu', padding='same'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  if hp.Boolean("dropout_1"):
    model.add(layers.Dropout(0.25))

  if hp.Boolean("conv_2"):
  # # Second conv block
    model.add(layers.Conv2D(filters=hp.Int("conv_3", min_value=40, max_value = 80, step = 8), kernel_size = (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(filters=hp.Int("conv_4", min_value=40, max_value = 80, step = 8), kernel_size = (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

  # Fully connected layers
  model.add(layers.Flatten())
  model.add(layers.Dense(units = hp.Int("dense_1", min_value=64, max_value=512, step=64), activation='relu'))
  model.add(layers.Dropout(rate = hp.Float("dropout_1", min_value=0.1, max_value=0.3, step=0.05)))
  model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

  model.summary()

  callback_model = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

  return model

In [ ]:
build_cnn(keras_tuner.HyperParameters())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_3 (RandomFlip)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_3               │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 40)     │         5,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 40)     │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 40)     │        14,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       163,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,722 (733.29 KB)

 Trainable params: 187,642 (732.98 KB)

 Non-trainable params: 80 (320.00 B)

<Sequential name=sequential_3, built=True>

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel = build_cnn,
    objective = "val_accuracy",
    max_trials = 3,
    executions_per_trial = 2,
    overwrite = True,
    directory = "cnn_model",
    project_name = "cnn_v1"
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_4 (RandomFlip)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_4               │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 40)     │         5,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 40)     │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 40)     │        14,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8, 8, 40)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       163,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,722 (733.29 KB)

 Trainable params: 187,642 (732.98 KB)

 Non-trainable params: 80 (320.00 B)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 48, 'step': 8, 'sampling': 'linear'}
conv_2 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 48, 'step': 8, 'sampling': 'linear'}
dropout_1 (Boolean)
{'default': False, 'conditions': []}
conv_3 (Int)
{'default': None, 'conditions': [], 'min_value': 40, 'max_value': 80, 'step': 8, 'sampling': 'linear'}
conv_4 (Int)
{'default': None, 'conditions': [], 'min_value': 40, 'max_value': 80, 'step': 8, 'sampling': 'linear'}
dense_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': 'linear'}


In [ ]:
# test / validate split
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    training_data["data"], training_data["labels"], test_size=0.2, shuffle=True
)

In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data = (x_val, y_val))

Trial 3 Complete [00h 00m 02s]

Best val_accuracy So Far: 0.4235000014305115
Total elapsed time: 00h 01m 20s


In [ ]:
models = tuner.get_best_models()
best_model = models[0]
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 56)     │        16,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 16, 56)     │           224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 72)     │        36,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 358,538 (1.37 MB)

 Trainable params: 358,426 (1.37 MB)

 Non-trainable params: 112 (448.00 B)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 56)     │        16,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 16, 56)     │           224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 72)     │        36,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 358,538 (1.37 MB)

 Trainable params: 358,426 (1.37 MB)

 Non-trainable params: 112 (448.00 B)

In [ ]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 56)     │        16,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 16, 56)     │           224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 72)     │        36,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 72)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 358,538 (1.37 MB)

 Trainable params: 358,426 (1.37 MB)

 Non-trainable params: 112 (448.00 B)

In [ ]:
callback_model = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)


history_cifar_cnn = best_model.fit(
    training_data["data"],
    training_data["labels"],
    batch_size = 1024,
    epochs=100,
    validation_split=0.1,
    callbacks=[callback_model]
    )

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 145ms/step - accuracy: 0.5016 - loss: 1.3588 - val_accuracy: 0.5348 - val_loss: 1.3376
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.5365 - loss: 1.2777 - val_accuracy: 0.5514 - val_loss: 1.2591
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.5456 - loss: 1.2430 - val_accuracy: 0.5676 - val_loss: 1.1994
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.5547 - loss: 1.2267 - val_accuracy: 0.5786 - val_loss: 1.1516
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.5719 - loss: 1.1869 - val_accuracy: 0.5844 - val_loss: 1.1426
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - accuracy: 0.5786 - loss: 1.1672 - val_accuracy: 0.5916 - val_loss: 1.1566
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.5865 - loss: 1.1457 - val_accuracy: 0.5836 - val_loss: 1.1612
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.5995 - loss: 1.1137 - val_accuracy: 